In [1]:
import torch
import pennylane as qml
from time import time
from tqdm import tqdm
import numpy as np
import matplotlib as plt
from circuit_model_II import QuantumCircuit, FullQuantumModel
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import os

In [2]:
num_qubits = 8
num_layers = 5
model = FullQuantumModel(num_qubits, num_layers)

# Dataset preparation

In [3]:
# Download MNIST and prepare transforms
mnist_train = datasets.MNIST(root='./data', train=True, download=True,
                             transform=transforms.Compose([
                                transforms.Resize((16, 16)),  # Resize to 16x16
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))  # Normalize
                             ]))
# Filter for zeros and ones
data = []
targets = []
for image, label in mnist_train:
    if label in [0, 1]:
        data.append(image.squeeze())
        targets.append(label)

data = torch.stack(data)
targets = torch.tensor(targets)
# Select 1024 zeros and 1024 ones for speed
zeros_indices = (targets == 0)
ones_indices = (targets == 1)

zeros = data[zeros_indices]
ones = data[ones_indices]

# take a subsample of the dataset for simplicity
zeros = zeros[:1024]
ones = ones[:1024]

#normalize between 0 and 1
zeros_max = torch.max(zeros.reshape(-1, 16*16), dim = 1)
zeros_min = torch.min(zeros.reshape(-1, 16*16), dim = 1)
ones_max = torch.max(ones.reshape(-1, 16*16), dim = 1)
ones_min = torch.min(ones.reshape(-1, 16*16), dim = 1)

def normalize(imgs):
  maxes, _ = torch.max(imgs.reshape(-1, 16*16), dim = 1)
  mins, _ = torch.min(imgs.reshape(-1, 16*16), dim = 1)

  mins = mins.unsqueeze(1).unsqueeze(2)
  maxes = maxes.unsqueeze(1).unsqueeze(2)

  return (imgs-mins)/(maxes-mins)

zeros = normalize(zeros)
ones = normalize(ones)

# assert images have min 0 and max 1 within an error of 1e-5
assert torch.allclose(zeros.min(), torch.tensor(0., dtype = torch.float32), atol=1e-5)
assert torch.allclose(zeros.max(), torch.tensor(1., dtype = torch.float32), atol=1e-5)
assert torch.allclose(ones.min(), torch.tensor(0., dtype = torch.float32), atol=1e-5)
assert torch.allclose(ones.max(), torch.tensor(1., dtype = torch.float32), atol=1e-5)

# concatenate the two datasets
zeros = zeros.flatten(start_dim = 1)
ones = ones.flatten(start_dim = 1)
dataset = torch.cat((zeros, ones), dim = 0)

# add labels
labels = torch.cat((torch.zeros((zeros.shape[0], 1)), torch.ones((ones.shape[0], 1))), dim = 0).squeeze()

# build dataloader
dataset = torch.utils.data.TensorDataset(dataset, labels)

BATCH_SIZE = 32

dataloader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)

In [4]:
type(dataloader.batch_size)

int

# Model training

In [6]:
type(model.fit(dataloader=dataloader, learning_rate=0.01, epochs=1, num_layers_to_execute = 2))

Epoch 1/1: 100%|██████████| 64/64 [00:01<00:00, 44.97it/s, accuracy=1, loss=0.403]    

Time per epoch:  1.4402949810028076
Epoch:  0 Loss:  0.40266889333724976
Accuracy:  1.0
--------------------------------------------------------------------------


tuple

In [5]:
for i in model.parameters():
    print(i) 

Parameter containing:
tensor([[0.1259, 0.3466, 0.9910],
        [0.8657, 0.9612, 0.4612],
        [0.7805, 0.8487, 0.4308],
        [0.0870, 0.7718, 0.3512],
        [0.9045, 0.2161, 0.2771],
        [0.4164, 0.4911, 0.4528],
        [0.0973, 0.9993, 0.6357],
        [0.1389, 0.3376, 0.3310]], requires_grad=True)
Parameter containing:
tensor([[0.8652, 0.8335, 0.5623],
        [0.7110, 0.6649, 0.1241],
        [0.2517, 0.6070, 0.3412],
        [0.5539, 0.5615, 0.3149],
        [0.0444, 0.3797, 0.1786],
        [0.5347, 0.1856, 0.1057],
        [0.1850, 0.3459, 0.6451],
        [0.1199, 0.0703, 0.8707]], requires_grad=True)
Parameter containing:
tensor([[0.1950, 0.7310, 0.8196],
        [0.3256, 0.8464, 0.1401],
        [0.5943, 0.8618, 0.8901],
        [0.1191, 0.2418, 0.3751],
        [0.9871, 0.5877, 0.6899],
        [0.0342, 0.7633, 0.5293],
        [0.5439, 0.8720, 0.9959],
        [0.6331, 0.0753, 0.7506]], requires_grad=True)
Parameter containing:
tensor([[0.5530, 0.6335, 0.8593],

In [7]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[-0.4217, -0.1460,  0.6143],
        [ 0.4643,  0.2922,  0.9641],
        [ 0.6993,  0.7714,  0.4006],
        [-0.3034,  1.3796,  0.7367],
        [ 1.2409,  0.0914,  0.0301],
        [-0.0668,  0.1032, -0.0940],
        [ 0.5682,  1.3219,  0.2524],
        [-0.3050, -0.0117,  1.0452]], requires_grad=True)
Parameter containing:
tensor([[ 0.8208,  0.8103,  0.5783],
        [ 0.4210,  0.2859,  0.1263],
        [-0.1307,  0.0789,  0.3703],
        [-0.0428,  0.0444,  0.3014],
        [-0.1649,  0.1395,  0.1812],
        [ 0.0893, -0.0510,  0.1188],
        [ 0.6903,  0.5038,  0.6650],
        [ 0.3607,  0.3417,  0.9038]], requires_grad=True)
Parameter containing:
tensor([[0.1950, 0.7310, 0.8196],
        [0.3256, 0.8464, 0.1401],
        [0.5943, 0.8618, 0.8901],
        [0.1191, 0.2418, 0.3751],
        [0.9871, 0.5877, 0.6899],
        [0.0342, 0.7633, 0.5293],
        [0.5439, 0.8720, 0.9959],
        [0.6331, 0.0753, 0.7506]])
Parameter containing:
tenso

In [10]:
model.unfreeze_layers([0,1])

In [18]:
torch.torch.linalg.norm(circuit(example), dim = 1)

tensor([1.0000], dtype=torch.float64)

In [17]:
§torch.torch.linalg.norm(example, dim = 1)

tensor([1.0000])

In [13]:
list(range(model.num_layers))

[0, 1, 2, 3, 4]

SyntaxError: invalid syntax (4209440489.py, line 1)

In [17]:
[element for element in range(model.num_layers) if element not in list(range(3))]


[3, 4]